# 0. Configuration Spark

remarque : on suit [le tp de formation à spark du ssplab](https://github.com/InseeFrLab/formation-spark/tree/main/5-hive-metastore)

# 1. lecture des données

In [123]:
import os
import s3fs
import json

from pyspark.sql.types import StructType



df = (spark.read
      .format("csv")
      .load("s3a://projet-si-collecte/json.txt")
     )

df.printSchema()

root
 |-- _c0: string (nullable = true)



Problème de schéma, le fichier n'est pas en format Json classique

In [100]:

f = open("json100000.txt","r")
lines = f.readlines()
data=[line.replace("'","\"") for line in lines]
data=[line.replace("True","\"True\"") for line in lines]

In [124]:

df = (spark.read
      .format("json")
     # .options(header='false', inferschema='false', delimiter="\n")
      .load("s3a://projet-si-collecte/json3.txt")
     )

In [125]:
df.printSchema()

root
 |-- COLLECTED: struct (nullable = true)
 |    |-- COMMENT: string (nullable = true)
 |    |-- PRODUCER: string (nullable = true)
 |    |-- READY: string (nullable = true)
 |-- EXTERNAL: struct (nullable = true)
 |    |-- LAST_BROADCAST: string (nullable = true)



In [ ]:
Le schéma est réparé. On peut mettre le fichier dans Hive.

# 2. mettre la BDD dans HIVE

In [129]:
spark.sql('show databases;').show()
# Pour l'instant on n'a pas de table SQL.

+---------+
|namespace|
+---------+
|  default|
+---------+



In [1]:
### On réutilise l'objet conf des autres tutoriels mais on lance la session spark avec le builder de sparksession pour activer le support de Hive
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import os
conf = SparkConf()

#url par défaut d'une api kubernetes accédé depuis l'intérieur du cluster (ici le notebook tourne lui même dans kubernetes)
conf.setMaster("k8s://https://kubernetes.default.svc:443")

#image des executors spark: pour des raisons de simplicité on réutilise l'image du notebook
conf.set("spark.kubernetes.container.image",  os.environ['IMAGE_NAME'])

# Nom du compte de service pour contacter l'api kubernetes : attention le package du datalab crée lui même cette variable d'enviromment.
# Dans un pod du cluster kubernetes il faut lire le fichier /var/run/secrets/kubernetes.io/serviceaccount/token
# Néanmoins ce paramètre est inutile car le contexte kubernetes local de ce notebook est préconfiguré
# conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) 

# Nom du namespace kubernetes
conf.set("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])

# Nombre d'executeur spark, il se lancera autant de pods kubernetes que le nombre indiqué.
conf.set("spark.executor.instances", "5")

# Mémoire alloué à la JVM
# Attention par défaut le pod kubernetes aura une limite supérieur qui dépend d'autres paramètres.
# On manipulera plus bas pour vérifier la limite de mémoire totale d'un executeur
conf.set("spark.executor.memory", "4g")

conf.set("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME'])


from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("5-Hive-metastore").config(conf = conf).enableHiveSupport().getOrCreate()


2022-06-15 09:53:48,983 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-06-15 09:53:51,583 WARN spark.ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


## 2.1 Récupérer le schéma de la base pour la requête sparkSQL

In [16]:
from pyspark import SQLContext
df = (spark.read
      .format("json")
     # .options(header='false', inferschema='false', delimiter="\n")
      .load("s3a://projet-si-collecte/json3.txt")
     )
df.printSchema()
cols = df.dtypes
buf = []
buf.append('CREATE EXTERNAL TABLE IF NOT EXISTS queen2 (')
keyanddatatypes =  df.dtypes
sizeof = len(df.dtypes)
count=1;
for eachvalue in keyanddatatypes:
    if count == sizeof:
        total = str(eachvalue[0])+str(' ')+str(eachvalue[1])
    else:
        total = str(eachvalue[0]) + str(' ') + str(eachvalue[1]) + str(',')
    buf.append(total)
    count = count + 1

buf.append(' )')
buf.append(' STORED as parquet ')
buf.append("LOCATION ")
buf.append("'")
buf.append('s3a://projet-si-collecte/table/data')
buf.append("'")

##partition by pt
tabledef = ''.join(buf)

print ("\n---------print definition ---------\n")
print (tabledef)

root
 |-- COLLECTED: struct (nullable = true)
 |    |-- COMMENT: string (nullable = true)
 |    |-- PRODUCER: string (nullable = true)
 |    |-- READY: string (nullable = true)
 |-- EXTERNAL: struct (nullable = true)
 |    |-- LAST_BROADCAST: string (nullable = true)


---------print definition ---------

CREATE EXTERNAL TABLE IF NOT EXISTS queen2 (COLLECTED struct<COMMENT:string,PRODUCER:string,READY:string>,EXTERNAL struct<LAST_BROADCAST:string> ) STORED as parquet LOCATION 's3a://projet-si-collecte/table/data'


# 2.2 Lancer la requête SQL pour créer la table

In [20]:
spark.sql(tabledef);

In [22]:
spark.sql("show tables").show();

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|    queen|      false|
|  default|   queen2|      false|
+---------+---------+-----------+



In [29]:
df.registerTempTable("data")

/opt/spark/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


# 3. La base est dans HIVE

# 4. Créer un trino


## 4.1 Configurer le trino

Décocher dans `security` le `Enable IP protection - Only the configured set of IPs will be able to reach the service` pour que trino soit accessible par superset

# 5. Créer un superset

## 5.1 charger la BDD

Aller dans le menu `Data` > `Datasets`

Cliquer en haut à droite sur `+ DATASET`

Choisir le bon `DATASET` (un qui commence par `hive-...`)

Choisir le bon `SCHEMA` (`default`)

Choisir la bonne `TABLE`

# 5.2 Créer le dashboard + chart

TODO : ajouter les étapes